# Chapter 8. 데이터 준비하기: 조인, 병합, 변형
- join: 두 개 데이터프레임의 공통 인덱스를 기준으로 두 개의 데이터프레임을 하나로 합침
- merge (병합) : 두 개 데이터프레임의 공통 열 또는 공통 인덱스를 기준으로 두 개의 데이터프레임을 하나로 합침 (how 속성 이용)
- concat(연결): 인덱스를 기준으로 연결

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 저자 깃허브 주소
path = 'https://raw.githubusercontent.com/wesm/pydata-book/refs/heads/3rd-edition/examples/'

In [3]:
df1 = pd.DataFrame({"key": ['a', 'b', 'c','f'],
                   "c1": [1,2,3,5]})

df2 = pd.DataFrame({"key": ['a', 'b', 'd','f'],
                   "c2": [5,6,7,8]})

df1

,key,c1
0,a,1
1,b,2
2,c,3
3,f,5


In [4]:
df2

,key,c2
0,a,5
1,b,6
2,d,7
3,f,8


### 병합 및 조인

In [5]:
# merge -  default는 inner (공통된 열에 있는 데이터만 병합)
df1.merge(df2)

,key,c1,c2
0,a,1,5
1,b,2,6
2,f,5,8


#### how 속성을 이용한 병합

In [6]:
# df1을 기준으로 병합
df1.merge(df2, how='left') # 왼쪽 기준

,key,c1,c2
0,a,1,5.0
1,b,2,6.0
2,c,3,NaN
3,f,5,8.0


In [7]:
df1.merge(df2, how='right') # 오른쪽 기준

,key,c1,c2
0,a,1.0,5
1,b,2.0,6
2,d,NaN,7
3,f,5.0,8


In [8]:
df1.merge(df2, how='outer') # 외부조인 (양쪽 모두)

,key,c1,c2
0,a,1.0,5.0
1,b,2.0,6.0
2,c,3.0,NaN
3,d,NaN,7.0
4,f,5.0,8.0


#### left_on, right_on 속성
- 키가 되는 기준열의 이름이 두 데이터프레임에서 다른 경우 사용

In [9]:
# 키가 서로 다른 데이터프레임 생성
df3 = pd.DataFrame({"key3": ['a', 'b', 'c','f'],
                   "c1": [1,2,3,5]})

df4 = pd.DataFrame({"key4": ['a', 'b', 'd','f'],
                   "c2": [5,6,7,8]})

df3

,key3,c1
0,a,1
1,b,2
2,c,3
3,f,5


In [10]:
df4

,key4,c2
0,a,5
1,b,6
2,d,7
3,f,8


In [11]:
# df3.merge(df4, how='left')
# 서로 키가 다르므로 병합이 안 되고 오류 발생
# MergeError:

# left_on이나 right_on 둘 중 하나라도 생략 시 에러 발생
df3.merge(df4, left_on='key3', right_on='key4') # 기본값은 공통된 열만 남는 내부 조인

,key3,c1,key4,c2
0,a,1,a,5
1,b,2,b,6
2,f,5,f,8


#### left_index, right_index

In [12]:
# left_index=True: df3의 인덱스를 기준으로 병합
# right_index=True: df4의 인덱스를 기준으로 병합
df3.merge(df4, left_index=True, right_index=True)
# 둘 중 하나라도 생략하면 MergeError: Must pass right_on or right_index=True

,key3,c1,key4,c2
0,a,1,a,5
1,b,2,b,6
2,c,3,d,7
3,f,5,f,8


#### concat (연결)

In [13]:
df1 = pd.DataFrame({'c1': [1,2,3,4], 'c2': [5,6,7,8]})
df1

,c1,c2
0,1,5
1,2,6
2,3,7
3,4,8


In [14]:
df2 = pd.DataFrame({'c3': ['a','b','c','d'],
                      'c4': [1.2,3.4,5.5,7.6]})
df2

,c3,c4
0,a,1.2
1,b,3.4
2,c,5.5
3,d,7.6


In [15]:
# 왼쪽에서 오른쪽으로 쌓음
# 인덱스 기준으로 같은 인덱스가 있을 때 쌓고, 없으면 NaN
pd.concat([df1, df2], axis=1)

,c1,c2,c3,c4
0,1,5,a,1.2
1,2,6,b,3.4
2,3,7,c,5.5
3,4,8,d,7.6


In [16]:
# 위에서 아래로 쌓음
# 칼럼 기준으로 같은 칼럼이 있을 때 쌓고, 없으면 NaN
pd.concat([df1, df2], axis=0)

,c1,c2,c3,c4
0,1.0,5.0,NaN,NaN
1,2.0,6.0,NaN,NaN
2,3.0,7.0,NaN,NaN
3,4.0,8.0,NaN,NaN
0,NaN,NaN,a,1.2
1,NaN,NaN,b,3.4
2,NaN,NaN,c,5.5
3,NaN,NaN,d,7.6


In [17]:
pd.concat([df1, df2]) # 기본값은 칼럼 기준

,c1,c2,c3,c4
0,1.0,5.0,NaN,NaN
1,2.0,6.0,NaN,NaN
2,3.0,7.0,NaN,NaN
3,4.0,8.0,NaN,NaN
0,NaN,NaN,a,1.2
1,NaN,NaN,b,3.4
2,NaN,NaN,c,5.5
3,NaN,NaN,d,7.6


#### reset_index
- 새로운 인덱스가 생기며, 원래 인덱스는 "index"라는 이름의 column으로 만듦.
- 기존 인덱스가 필요 없을 경우 drop 옵션 사용(drop=True)

In [18]:
# reset_index
df_result = pd.concat([df1, df2])
df_result.reset_index()

,index,c1,c2,c3,c4
0,0,1.0,5.0,NaN,NaN
1,1,2.0,6.0,NaN,NaN
2,2,3.0,7.0,NaN,NaN
3,3,4.0,8.0,NaN,NaN
4,0,NaN,NaN,a,1.2
5,1,NaN,NaN,b,3.4
6,2,NaN,NaN,c,5.5
7,3,NaN,NaN,d,7.6


In [19]:
df_result.reset_index(drop=True) # 원래 인덱스 삭제 후 새로운 인덱스 생성

,c1,c2,c3,c4
0,1.0,5.0,NaN,NaN
1,2.0,6.0,NaN,NaN
2,3.0,7.0,NaN,NaN
3,4.0,8.0,NaN,NaN
4,NaN,NaN,a,1.2
5,NaN,NaN,b,3.4
6,NaN,NaN,c,5.5
7,NaN,NaN,d,7.6


#### 원본 변경 적용 여부
- inpalce=True 를 사용하면 변경된 작업이 원본에도 적용된다

In [20]:
df_result

,c1,c2,c3,c4
0,1.0,5.0,NaN,NaN
1,2.0,6.0,NaN,NaN
2,3.0,7.0,NaN,NaN
3,4.0,8.0,NaN,NaN
0,NaN,NaN,a,1.2
1,NaN,NaN,b,3.4
2,NaN,NaN,c,5.5
3,NaN,NaN,d,7.6


In [21]:
df_result.reset_index(drop=True, inplace=True)
df_result

,c1,c2,c3,c4
0,1.0,5.0,NaN,NaN
1,2.0,6.0,NaN,NaN
2,3.0,7.0,NaN,NaN
3,4.0,8.0,NaN,NaN
4,NaN,NaN,a,1.2
5,NaN,NaN,b,3.4
6,NaN,NaN,c,5.5
7,NaN,NaN,d,7.6


### 정렬
- 데이터프레임 정렬: sort_index() 또는 sort_values()

#### 행 이름으로 정렬

In [22]:
# iris 데이터셋 불러오기 (시본 라이브러리)
import seaborn as sns
iris = sns.load_dataset('iris')
print(len(iris))
iris.head()

150


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [23]:
iris.sort_index(ascending=False).head() # 내림차순

,sepal_length,sepal_width,petal_length,petal_width,species
149,5.9,3.0,5.1,1.8,virginica
148,6.2,3.4,5.4,2.3,virginica
147,6.5,3.0,5.2,2.0,virginica
146,6.3,2.5,5.0,1.9,virginica
145,6.7,3.0,5.2,2.3,virginica


In [24]:
iris.sort_index(axis=1).head()

,petal_length,petal_width,sepal_length,sepal_width,species
0,1.4,0.2,5.1,3.5,setosa
1,1.4,0.2,4.9,3.0,setosa
2,1.3,0.2,4.7,3.2,setosa
3,1.5,0.2,4.6,3.1,setosa
4,1.4,0.2,5.0,3.6,setosa


In [25]:
iris.sort_index(axis=1,inplace=True)
iris.head()

,petal_length,petal_width,sepal_length,sepal_width,species
0,1.4,0.2,5.1,3.5,setosa
1,1.4,0.2,4.9,3.0,setosa
2,1.3,0.2,4.7,3.2,setosa
3,1.5,0.2,4.6,3.1,setosa
4,1.4,0.2,5.0,3.6,setosa


In [26]:
# 원본 데이터
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [27]:
# sort_values: 값 기준으로 정렬
# by: 기준
# iris.sort_values(by=['sepal_length'], inplace=True)

# 기준 우선 순위 지정 (sepal_length > 동률일 경우 sepal_width 기준)
iris.sort_values(by=['sepal_length','sepal_width'], inplace=True)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
13,4.3,3.0,1.1,0.1,setosa
8,4.4,2.9,1.4,0.2,setosa
38,4.4,3.0,1.3,0.2,setosa
42,4.4,3.2,1.3,0.2,setosa
41,4.5,2.3,1.3,0.3,setosa


In [28]:
iris.reset_index(drop=True, inplace=True)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.3,3.0,1.1,0.1,setosa
1,4.4,2.9,1.4,0.2,setosa
2,4.4,3.0,1.3,0.2,setosa
3,4.4,3.2,1.3,0.2,setosa
4,4.5,2.3,1.3,0.3,setosa


In [29]:
# 원본 데이터
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [30]:
# level 지정

# 2차원 리스트 생성
iris.columns = [['sepal', 'sepal', 'petal', 'petal', 'species'], # level0
                iris.columns] # level 1

iris.columns.names = ['info', 'details'] # info: level0 이름 / details: level0에 대한 이름
iris.head()

info           sepal                    petal             species
details sepal_length sepal_width petal_length petal_width species
0                5.1         3.5          1.4         0.2  setosa
1                4.9         3.0          1.4         0.2  setosa
2                4.7         3.2          1.3         0.2  setosa
3                4.6         3.1          1.5         0.2  setosa
4                5.0         3.6          1.4         0.2  setosa

In [31]:
# 레벨 기준 정렬
iris.sort_index(level=["info"], axis=1).head()

info           petal                    sepal             species
details petal_length petal_width sepal_length sepal_width species
0                1.4         0.2          5.1         3.5  setosa
1                1.4         0.2          4.9         3.0  setosa
2                1.3         0.2          4.7         3.2  setosa
3                1.5         0.2          4.6         3.1  setosa
4                1.4         0.2          5.0         3.6  setosa

In [32]:
# axis=1 생략 시, 데이터를 인덱스에서 찾으므로 에러 발생 (AssertionError: )
# iris.sort_index(level=["info"], ascending=False).head()

iris.sort_index(level=["info"], axis=1, ascending=False).head()

info    species       sepal                    petal             
details species sepal_width sepal_length petal_width petal_length
0        setosa         3.5          5.1         0.2          1.4
1        setosa         3.0          4.9         0.2          1.4
2        setosa         3.2          4.7         0.2          1.3
3        setosa         3.1          4.6         0.2          1.5
4        setosa         3.6          5.0         0.2          1.4

In [33]:
# level을 번호로 주는 것도 가능
# 가장 바깥: level0, 안으로 들어올 수록 level 커짐
iris.sort_index(level=0, axis=1, ascending=False).head()

info    species       sepal                    petal             
details species sepal_width sepal_length petal_width petal_length
0        setosa         3.5          5.1         0.2          1.4
1        setosa         3.0          4.9         0.2          1.4
2        setosa         3.2          4.7         0.2          1.3
3        setosa         3.1          4.6         0.2          1.5
4        setosa         3.6          5.0         0.2          1.4

## 8.1 계층적 색인
- 판다스의 중요한 기능으로, 축에 대해 둘 이상의 색인 단계 지정 가능
- 차원 수가 높은 데이터를 낮은 차원의 형식으로 다룰 수 있게 해주는 기능
- 계층적 색인은 데이터를 재구성하고 피벗 테이블 생성 같은 그룹 기반으로 작업할 때 중요하게 사용

In [34]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

data # MultiIndex를 색인으로하는 Series

a  1    0.640755
   2    0.492451
   3    0.364224
b  1    0.872882
   3    0.512695
c  1    0.027550
   2    0.986360
d  2    0.913030
   3    0.967229
dtype: float64

In [35]:
# 바로 위 단계의 색인을 이용해 하위 계층에 접근하기
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [36]:
# 데이터의 부분집합을 부분적 색인으로 접근하기
data["b"]

,0
1,0.872882
3,0.512695


In [37]:
data["b": "c"]

b  1    0.872882
   3    0.512695
c  1    0.027550
   2    0.986360
dtype: float64

In [38]:
data.loc[["b", "d"]]

b  1    0.872882
   3    0.512695
d  2    0.913030
   3    0.967229
dtype: float64

In [39]:
# 하위 계층의 객체 선택하기
data.loc[:, 2]

,0
a,0.492451
c,0.986360
d,0.913030


In [40]:
# unstack 메서드를 사용해 데이터를 새롭게 배열
data.unstack()

,1,2,3
a,0.640755,0.492451,0.364224
b,0.872882,NaN,0.512695
c,0.027550,0.986360,NaN
d,NaN,0.913030,0.967229


In [41]:
# stack은 unstack의 반대
data.unstack().stack()

a  1    0.640755
   2    0.492451
   3    0.364224
b  1    0.872882
   3    0.512695
c  1    0.027550
   2    0.986360
d  2    0.913030
   3    0.967229
dtype: float64

In [42]:
# DataFrame 에서는 두 축 모두 계층적 색인 가질 수 있음
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a","a","b","b"], [1, 2, 1, 2]],
                     columns = [["Ohio", "Ohio", "Colorado"],
                                ["Green", "Red", "Green"]])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [43]:
# 계층적 색인의 각 단계에 이름(문자열이나 어떤 파이썬 객체라도 가능) 설정 가능
# 만약 이름을 가질 시, 콘솔 출력 시 함께 나타남
frame.index.names = ["key1", "key2"]

frame.columns.names = ["state", "color"]

frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [44]:
# nlevels: 색인이 몇 개의 계층을 갖는지 확인
frame.index.nlevels

2

In [45]:
# 열의 부분집합을 부분적 색인으로 접근하기
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [46]:
# MultiIndex는 따로 생성한 다음 재사용 가능
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green", "Red", "Green"]],
                          names = ["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### 8.1.1 계층의 순서를 바꾸고 정렬하기
- swaplevel: 넘겨받은 두 개의 계층 번호나 이름이 뒤바뀐 새로운 객체 반환 (데이터는 변경되지 않음)
- sort.index: 모든 색인 게층을 사용해 사전 순으로 데이터 정렬
> level 인수를 이용해 단일 계층만 사용하거나 일부 계층만 선택해서 정렬 가능

In [47]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [48]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [49]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 8.1.2 계층별 요약 통계
- DataFrame이나 Series 의 많은 요약 통계, 기술 통계는 level 옵션을 가짐
- 어떤 하나의 축에 대해 합을 구하고 싶은 단계를 지정하는 옵션

In [50]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [51]:
frame.groupby(level="color", axis="columns").sum()

<ipython-input-51-0b8b7712eee1>:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.3 DataFrame의 열 사용하기
- set_index: 하나 이상의 열을 색인으로 하는 DatFrame 생성
- set_index 메서드에 drop=False를 명시적으로 지정하지 않으면 색인으로 지정한 열은 DataFrame에서 삭제됨
- reset_index: set_index와 반대되는 개념으로, 계층적 색인 단계가 열로 이동

In [52]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [53]:
# set_index
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [54]:
frame2 = frame.set_index(["c", "d"], drop=False)
frame2

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [55]:
 # reset_index
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})

frame2 = frame.set_index(["c", "d"])

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 데이터 합치기
- pandas.merge: 하나 이상의 키를 기준으로 DataFrame 의 행을 연결
- pandas.concat: 하나의 축을 따라 객체를 이어 붙임
- combine_first: 두 객체를 겹쳐서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채움

### 8.2.1 데이터베이스 스타일로 DataFrame 합치기
- merge 함수는 기본적으로 내부 조인을 수행해 교집합 결과 반환
- merge 함수의 how에 inner, left, right, outer 를 넘기면 조인 방식 설정 가능 (기본값 inner)
- inner > 내부 조인: 양쪽 테이블 모두에 존재하는 키 조합 사용
- outer > 외부 조인: 합집합 결과 반환
- left > 왼쪽 조인: 왼쪽의 모든 행 포함하는 결과 반환
- right > 오른쪽 조인: 오른쪽의 모든 행 포함하는 결과 반환
- merge 메서드의 종류에 따라 어떤 키 조합이 결과로 반환되는지 알고 싶다면 여러 개의 키가 들어 있는 튜플의 배열이 단일 조인 키로 사용된다고 생각하면 된다
- 열과 열을 조인할 때 전달한 DataFrame 객체의 색인은 무시
- 만약 색인값을 유지해야 한다면 reset_index를 이용해서 해당 열에 색인 추가

#### pandas.merge 함수의 인수
- left: 병합하려는 DataFrame 중 왼쪽에 위치한 DataFrame
- right: 병합하려는 DataFrame 중 오른쪽에 위치한 DataFrame
- how: 조인 방법. inner(기본값), outer, left, right
- on: 조인하려는 열 이름. 반드시 두 DataFrame 객체 모두에 존재하는 이름이어야 함.
> 만약 명시하지 않고 다른 조인 키도 주어지지 않으면 left와 right 사이에서 공통되는 열을 조인 키로 사용
- left_on: 조인 키로 사용할 left DataFrame의 열
- right_on: 조인 키로 사용할 right DataFrame의 열
- left_index: 조인 키로 사용할 left DataFrame의 색인 행(다중 색인일 경우 키)
- right_index: 조인 키로 사용할 right DataFrame의 색인 행(다중 색인일 경우 키)
- sort: 조인 키에 따라 병합된 데이터를 사전 순으로 정렬. 기본값 True.
> 대용량 데이터의 경우 False로 하면 성능상 이득 얻을 수 있음.
- suffixes: 열 이름이 겹칠 경우 각 열 이름 뒤에 붙일 문자열의 튜플. 기본값은 ("_x", "_y")
> 만약 data라는 열 이름이 양쪽 DataFrame에 같이 존재하면 결과에서는 data_x, data_y로 보여짐
- copy: False일 경우 예외적인 경우에 데이터가 결과로 복사되지 않도록 함. 기본값은 복사가 항상 이뤄짐.
- validate: 병합이 일대일, 일대다, 다대다 중 지정된 유형인지 확인.
- indicator: merge라는 이름의 특별한 열을 추가해 각 행의 소스가 어디인지 나타냄.
> left_only, right_only, both 값 가짐

In [56]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [57]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [58]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [59]:
# 다대일 조인
# df1 데이터는 key 열에 여러 개의 a,b 가짐
# df2 key 열은 유일한 행 가짐

# merge 함수 호출
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [60]:
# 명시적으로 지정하는 것이 좋음
pd.merge(df1, df2, on="key")
# pd.merge 연산 후 열 순서는 보장되지 않음

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [61]:
# 두 객체에 공통되는 열 이름이 하나도 없을 경우에는 별도로 지정
df1 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
pd.merge(df1, df2, left_on="lkey", right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [62]:
# how 옵션
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [63]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [64]:
# 다대다 병합: 일치하는 키에 대한 데크르트 곱 (곱집합) 생성
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})

In [65]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [66]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [67]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [68]:
# 여러 개의 키 병합을 위해선 열 이름이 담긴 리스트 넘김
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2":["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                     "key2":["one", "one", "one", "two"],
                     "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [69]:
# 겹치는 열 이름을 처리하는 방식
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [70]:
# 방법1: 축 이름을 변경하여 수동으로 열 이름 겹치게 하기
# 방법2: pandas.merge 함수의 suffixes 옵션으로 두 DataFrame 객체에서 겹치는 열 이름 뒤에 붙일 문자열 지정
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 8.2.2 색인 병합하기
- 병합하려는 키가 DataFrame의 색인행 레이블일 경우에는 left_inde=True 또는 right_index=True 옵션으로 해당 색인을 병합 키로 사용 가능
- join 메서드: 색인으로 병합 시 편리. join 메서드는 열이 겹치지 않으며 완전히 같거나 유사한 색인 구조를 가진 여러 개의 DataFrame 객체를 병합할 때 사용
- join 메서드는 기본적으로 왼쪽 조인 수행

In [71]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype='Int64')})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [72]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [73]:
right1

,group_val
a,3.5
b,7.0


In [74]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [75]:
# 외부 조인으로 합집합 구하기
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [76]:
# 계층적으로 색인된 데이터
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                      "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype='Int64')})

righth_index = pd.MultiIndex.from_arrays(
    [
        ["Navada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype='Int64', index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype='Int64', index=righth_index)
                       })

In [77]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [78]:
righth

event1  event2
Navada 2001       0       1
Nevada 2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [79]:
# 계층적으로 색인된 데이터는 리스트로 여러 개의 열을 지정해서 병합해야 함
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


In [80]:
# 중복되는 색인값 다룰 때는 how="outer" 옵션 사용
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
4,Navada,2001,<NA>,0,1
4,Nevada,2000,<NA>,2,3
3,Nevada,2001,3,<NA>,<NA>
4,Nevada,2002,4,<NA>,<NA>
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


In [81]:
# 양쪽에 공통으로 존재하는 여러 개의 색인 병합하기
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                     index=["b", "c", "d", "e"],
                     columns=["Missouri", "Alabama"]).astype("Int64")

In [82]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [83]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [84]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [85]:
# join 메서드
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [86]:
# join 메서드는 기본적으로 왼쪽 조인 수행하나, DataFrame 열 중 하나에 대해 조인 수행 가능
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [87]:
# 색인 대 색인으로 병합하기
# 병합하려는 DataFrame의 리스트를 join 메서드에 넘기면 됨
# 보통 pandas.concat 사용
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                     index=["a", "c", "e", "f"],
                     columns=["New York", "Oregon"])

another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [88]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [89]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


### 8.2.3 축 따라 이어 붙이기
[결합을 위한 함수]
- 넘파이: ndarray를 연결하는 concatenate 함수
- 판다스 concat 함수: axis="index"를 기본값으로 하며 새로운 Series 객체 생성
> 만약 axis="columns"로 넘기면 결과는 DataFrame

###[pandas.concat 함수 인수]
- objs: 이어 붙일 판다스 객체의 딕셔너리나 리스트. 필수 인수
- axis: 이어 붙일 축 방향. 행을 따라 연결하며 기본값은 axis="index"
- join: 조인 방식 inner(내부), outer(외부) 가 있으며, 기본값은 inner
- keys: 이어 붙일 객체나 이어 붙인 축에 대한 게층 색인을 생성하는 데 연관된 값. 리스트나 임의의 값이 들어 있는 배열, 튜플의 배열 또는 배열의 리스트 (levels 옵션에 다차원 배열이 넘어온 경우)
- names: keys나 levels 혹은 둘 다 있을 경우 생성된 게층 레벨을 위한 이름
- verify_intergrity: 이어 붙인 객체에 중복되는 축이 있는지 검사하고 있다면 예외 발생. 기본값은 False로 중복 허용
- ignore_index: 이어 붙인 축의 색인을 유지하지 않고 range(total_length)로 새로운 색인 생성

In [90]:
# 넘파이 concatenate 함수
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [91]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [92]:
# 판다스 concat 함수

# 겹치지 않는 세 개의 Series 객체
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")

s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")

s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [93]:
s1

,0
a,0
b,1


In [94]:
s2

,0
c,2
d,3
e,4


In [95]:
s3

,0
f,5
g,6


In [96]:
pd.concat([s1, s2, s3]) # Series 반환

,0
a,0
b,1
c,2
d,3
e,4
f,5
g,6


In [97]:
pd.concat([s1, s2, s3], axis="columns") # DataFrame 반환

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [98]:
s4 = pd.concat([s1, s3])

s4

,0
a,0
b,1
f,5
g,6


In [99]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [100]:
pd.concat([s1, s4], axis="columns", join="inner") # join="inner" 옵션으로 교집합 가능

,0,1
a,0,0
b,1,1


In [101]:
# 계층적 색인을 생성하려면 keys 인수 사용
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [102]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


In [103]:
# Series를 axis="columns"로 병합 시 keys는 DataFrame의 열 제목이 됨
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [104]:
# DataFrame에도 동일하게 적용
df1 = pd.DataFrame(np.arange(6).reshape((3, 2)), index=["a", "b", "c"],
                   columns=["one", "two"])

df2 = pd.DataFrame(5 + np.arange(4).reshape((2, 2)), index=["a", "c"],
                   columns=["three", "four"])

In [105]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [106]:
df2

,three,four
a,5,6
c,7,8


In [107]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [108]:
# 리스트 대신 객체의 딕셔너리를 넘기면 keys 옵션으로 딕셔너리 키 사용
pd.concat({"level1": df1, "level2": df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [109]:
# 새로 생성된 계층의 이름은 names 인수를 통해 지정
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
                                            names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [110]:
# DataFrame의 행 색인이 분석에 필요한 데이터를 포함하고 있지 않은 경우에는
# ignore_index=True 옵션 사용
# 각 DataFrame의 색인은 무시, 열에 있는 데이터만 이어 붙인 다음 새로운 기본 색인 할당
pd.concat([df1, df2], ignore_index=True)

,one,two,three,four
0,0.0,1.0,NaN,NaN
1,2.0,3.0,NaN,NaN
2,4.0,5.0,NaN,NaN
3,NaN,NaN,5.0,6.0
4,NaN,NaN,7.0,8.0


### 8.2.4 겹치는 데이터 합치기
- 배열 기반 if-else 구문과 동일한 기능을 하는 넘파이의 where 함수
- numpy.where를 사용하면 색인 레입르의 정렬 여부 확인 하지 않음.
- numpy.where 사용 시 길이가 동일할 필요 없음.

In [111]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])

b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5],
              index=["f", "e", "d", "c", "b", "a"])

In [112]:
a

,0
f,NaN
e,2.5
d,0.0
c,3.5
b,4.5
a,NaN


In [113]:
b

,0
f,0.0
e,NaN
d,2.0
c,NaN
b,NaN
a,5.0


In [114]:
np.where(pd.isna(a), b, a) # a값이 null일 때 b가 선택되고 그렇지 않으면 null이 아닌 a 값 선택

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [115]:
# Series의 combine_first: 색인에 따라 값 정렬 시
a.combine_first(b)

,0
f,0.0
e,2.5
d,0.0
c,3.5
b,4.5
a,5.0


In [116]:
# combine_first 메서드는 열에 대해 같은 작업 수행
# 호출하는 객체에서 누락된 데이터를 인수로 넘긴 객체에 있는 값으로 채워넣기 가능
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)
                    })

df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.],
                    })

In [117]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [118]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [119]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 재구성과 피벗
- 재구성 (피벗 연산): 표 형식의 데이터를 재배치하는 다양한 기본 연산

### 8.3.1 계층적 색인으로 재구성하기
- stack: 데이터의 열을 행으로 피벗 (또는 회전)
- unstack: 행을 열로 피벗


In [120]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"
                    ))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [121]:
# stack 메서드 사용하면 열이 행으로 피벗
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [122]:
# unstack 메서드를 사용하면 계층적 색인을 가진 Series 로부터 다시 DataFrame 얻을 수 있음
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [123]:
# level 이름이나 숫자를 전달해서 끄집어낼 단계 지정
# 기본적으로 가장 안쪽부터 끄집어 냄
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [124]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [125]:
# 해당 단계에 있는 모든 값이 하위 그룹에 속하지 않을 경우 누락된 데이터 발생
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")

s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")

data2 = pd.concat([s1, s2], keys=["one", "two"])

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [126]:
# stack 메서드는 누락된 데이터를 자동으로 걸러냄
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [127]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [128]:
data2.unstack().stack(dropna=False)

<ipython-input-128-03741fdba496>:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [129]:
# DataFrame을 unstack 하면 unstack은 결과에서 가장 낮은 단계가 됨
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))

df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [130]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [131]:
# stack 호출 시에도 쌓을 축의 이름 지정 가능
df.unstack(level="state").stack(level="side")

<ipython-input-131-3105b1b911f9>:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.unstack(level="state").stack(level="side")


state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

### 8.3.2 긴 형식에서 넓은 형식으로 피벗하기
- 시계열 데이터를 저장하는 일반적인 방법: 시간 순서대로 나열 (긴 형식 또는 적층 형식)
- pandas.PeriodIndex; 특정한 시간이 아닌 시간 간격을 나타내는 함수
- pivot 메서드: 데이터의 열을 기준으로 피벗테이블로 변환시키는 메서드
- pivot 메서드의 처음 두 값은 각각 행과 열 색인으로 사용할 열 이름이고 마지막 value는 DataFrame에 채워 넣을 값을 담을 열
- pivot 메서드는 set_index를 사용해서 계층적 색인을 만들고 unstack 메서드로 형태를 변경하는 것과 동일

In [132]:
# 시계열 데이터 다루기
data = pd.read_csv(path + 'macrodata.csv')

data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [133]:
# pandas.PeriodIndex를 이용해서 연도(year)와 분기(quarter) 열 합치기
# 날짜/시간 값(datetime)을 포함하도록 색인으로 설정
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date"
                         )

periods

<ipython-input-133-a81e195e44ad>:3: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.pop("year"),


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [134]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [135]:
# 행의 일부부분을 선택하고 열 색인에 "item" 이라는 이름 지정
data = data.reindex(columns=["realgdp", "infl", "unemp"])

data.columns.name="item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [136]:
# stack을 이용헤서 재구성
# reset_index를 사용해 새로운 색인 계층을 열로 바꿈
# 데이터 값을 담고 있는 열의 이름을 "value"로 지정
long_data = (data.stack()
            .reset_index()
            .rename(columns={0: "value"}))
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [137]:
# pivot 메서드 (행, 열, DataFrame에 채워넣을 value)
pivoted = long_data.pivot(index="date", columns="item", values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [138]:
# 한 번에 두 개의 열 변형
long_data.index.name = None
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,-0.624941
1,1959-01-01,infl,0.000,0.312308
2,1959-01-01,unemp,5.800,-0.612632
3,1959-04-01,realgdp,2778.801,0.151940
4,1959-04-01,infl,2.340,1.045850
5,1959-04-01,unemp,5.100,-1.143724
6,1959-07-01,realgdp,2775.488,0.018429
7,1959-07-01,infl,2.740,0.169737
8,1959-07-01,unemp,5.300,-0.411899
9,1959-10-01,realgdp,2785.204,0.720086


In [139]:
# 마지막 인수를 생략해서 계층적 열을 갖는 DataFrame 얻기
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.312308 -0.624941 -0.612632
1959-04-01  2.34  2778.801   5.1  1.045850  0.151940 -1.143724
1959-07-01  2.74  2775.488   5.3  0.169737  0.018429 -0.411899
1959-10-01  0.27  2785.204   5.6  0.328782  0.720086 -0.064293
1960-01-01  2.31  2847.699   5.2 -2.718182 -0.713411  0.162916

In [140]:
# pivot 메서드는 set_index를 사용해서 계층적 색인을 만들고 unstack 메서드로 형태를 변경하는 것과 동일
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.312308 -0.624941 -0.612632
1959-04-01  2.34  2778.801   5.1  1.045850  0.151940 -1.143724
1959-07-01  2.74  2775.488   5.3  0.169737  0.018429 -0.411899
1959-10-01  0.27  2785.204   5.6  0.328782  0.720086 -0.064293
1960-01-01  2.31  2847.699   5.2 -2.718182 -0.713411  0.162916

### 8.3.3 넓은 형식에서 긴 형식으로 피벗하기
- pandas.melt: pivot과 반대되는 연산으로, 하나의 열을 여러 개의 새로운 DataFrame으로 생성하지 않고, 여러 열을 하나로 병합해 입력보다 긴 DataFrame을 만듦
- pandas.melt를 사용할 때는 반드시 그룹 구분자로 사용할 열을 지정해야 함.
- pandas.melt는 pandas.pivot과 달리 그룹 구분자 없이도 사용 가능

In [141]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]
                   })

df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [142]:
# pandas.melt를 사용할 때는 반드시 그룹 구분자로 사용할 열을 지정해야 함.
# key를 그룹 구분자로 지정
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [ ]:
# pivot을 사용해서 원래 모양으로 되돌리기
reshaped = melted.pivot(index="key", columns="variable", values="value")
reshaped # 데이터 중복이 없거나, 집계를 하지 않을 경우 사용

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [ ]:
# pivot의 결과는 행 레이블로 사용하던 열에서 색인 생성하므로
# reset_index를 이용해서 데이터를 다시 열로 돌려놓음.
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [ ]:
# 데이터 값으로 사용할 열의 부분집합 지정하기
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [ ]:
# pd.melt는 그룹 구분자 없이도 가능
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [ ]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
